In [1]:
import pandas as pd

In [2]:
# Parameters
nb_name = "4b_merge_contract_omie_out"
config_path = "/Users/mikelperez/0_Python_projects_v3_reduced/0_Python_projects_reduced/configs/config.yaml"
tarifa = "6.1 TD"
path_contract_matrix = "/Users/mikelperez/0_Python_projects_v3_reduced/0_Python_projects_reduced/data/customers/elec/Generic/raw/contract_matrix_TRIAL.pkl"
path_omie_matrix = "/Users/mikelperez/0_Python_projects_v3_reduced/0_Python_projects_reduced/data/verification_project/elec/omie_matrix_verif.pkl"
path_master_matrix = "/Users/mikelperez/0_Python_projects_v3_reduced/0_Python_projects_reduced/data/customers/elec/Generic/raw/master_matrix_TRIAL.pkl"


In [4]:
df_contract_matrix = pd.read_pickle(path_contract_matrix)
df_omie_matrix = pd.read_pickle(path_omie_matrix)

df_contract_matrix = df_contract_matrix.copy()
df_contract_matrix['date'] = df_contract_matrix['datetime'].dt.date
df_contract_matrix.drop('datetime', axis=1, inplace=True)
df_omie_matrix['date'] = df_omie_matrix['datetime'].dt.date
df_omie_matrix

,datetime,periodo,value,ssaa,20_td,30_td,61_td,62_td,63_td,64_td,date
0,2023-01-01 00:00:00,6,0.00,26.761640,0.237,0.237,0.101,0.071,0.058,0.022,2023-01-01
1,2023-01-01 01:00:00,6,0.00,27.891910,0.247,0.247,0.105,0.074,0.060,0.023,2023-01-01
2,2023-01-01 02:00:00,6,0.00,38.725701,0.261,0.261,0.112,0.078,0.064,0.025,2023-01-01
3,2023-01-01 03:00:00,6,0.00,38.365772,0.272,0.272,0.116,0.081,0.066,0.026,2023-01-01
4,2023-01-01 04:00:00,6,0.00,40.265236,0.278,0.278,0.119,0.083,0.068,0.026,2023-01-01
...,...,...,...,...,...,...,...,...,...,...,...
21161,2025-05-31 19:00:00,6,31.78,17.172741,0.180,0.180,0.077,0.054,0.044,0.017,2025-05-31
21162,2025-05-31 20:00:00,6,62.00,15.426091,0.177,0.177,0.076,0.053,0.043,0.017,2025-05-31
21163,2025-05-31 21:00:00,6,64.31,16.026816,0.179,0.179,0.077,0.054,0.044,0.017,2025-05-31
21164,2025-05-31 22:00:00,6,74.44,14.124158,0.182,0.182,0.078,0.055,0.044,0.017,2025-05-31


In [5]:
print(f"Merging contract data to OMIE matrix")
df_master = pd.merge(df_omie_matrix, df_contract_matrix, on=['date', 'periodo'], how='left')
df_master.drop('date', axis=1, inplace=True)
# quedarme con las perdidas de la tarifa correspondiente

In [7]:
for col in df_master.columns:
    if '_td' in col:
        if (tarifa[0] != col[0]) or (tarifa [2] != col[1]):
            df_master.drop(col, axis=1, inplace=True)
        elif (tarifa[0] == col[0]) or (tarifa [2] == col[1]):
            df_master.rename(columns={f'{col}':'perd'}, inplace=True)

In [9]:
df_master.to_pickle(path_master_matrix)
print(f"Master matrix saved to {path_master_matrix}")